In [1]:
import pandas as pd
from pulp import *
import re

In [2]:
data = pd.read_csv("./data/IPL-aggregated-stats_final.csv")

In [3]:
data.head()

,PLAYER,Pts_sum,Pts_mean,years_played,Mat_sum,Mat_mean,Wkts_sum,Wkts_mean,Dots_sum,Dots_mean,...,Catches_sum,Catches_mean,Stumpings_sum,Stumpings_mean,position,median_season_ratings,avg_season_ratings,base_price,player_name,overseas_or_indian
0,ABdeVilliers,1712.5,155.681818,11,139,12.636364,0,0.0,0,0.0,...,80,7.272727,7,0.636364,BAT,80.344828,80.319112,1.5,ABdeVilliers,Overseas
1,ABdeVilliers,1712.5,155.681818,11,139,12.636364,0,0.0,0,0.0,...,80,7.272727,7,0.636364,BAT,80.344828,80.319112,1.5,ABdeVilliers,Overseas
2,ABdeVilliers,1712.5,155.681818,11,139,12.636364,0,0.0,0,0.0,...,80,7.272727,7,0.636364,BAT,80.344828,80.319112,1.5,ABdeVilliers,Overseas
3,ABdeVilliers,1712.5,155.681818,11,139,12.636364,0,0.0,0,0.0,...,80,7.272727,7,0.636364,BAT,80.344828,80.319112,1.5,ABdeVilliers,Overseas
4,ABdeVilliers,1712.5,155.681818,11,139,12.636364,0,0.0,0,0.0,...,80,7.272727,7,0.636364,BAT,80.344828,80.319112,1.5,ABdeVilliers,Overseas


In [4]:
data.overseas_or_indian.value_counts()

Indian      923
Overseas    560
Name: overseas_or_indian, dtype: int64

### The objective function along with other constraints are mentioned below

$$ max \sum_{players =1}^N rating$$
s.t $$ min-batsman <  chosen-batsman < max-batsman $$
s.t $$ min-batsman <  chosen-batsman < max-batsman $$
s.t $$ min-bowlers <  chosen-bowlers < max-bowlers $$
s.t $$ min-allrounders <  chosen-all-rounders < max-allrounders $$
s.t $$ wicketkeepers = 1 $$

### Creating the set of players that will go into the auction

In [5]:
players = list(data["PLAYER"])
indians = list(data[data["overseas_or_indian"]=="Indian"]["PLAYER"])
overseas = list(data[data["overseas_or_indian"]=="Overseas"]["PLAYER"])
batsman = list(data[data.position=="BAT"]["PLAYER"])
bowlers = list(data[data.position=="BWL"]["PLAYER"])
allrounders = list(data[data.position=="AR"]["PLAYER"])
wks = list(data[data.position=="WK"]["PLAYER"])

In [6]:
median_ratings = dict(zip(players, list(data["median_season_ratings"])))
base_prices = dict(zip(players, list(data["base_price"])))

### Lets implement a Binary Integer Programming strategy

In [7]:
prob = LpProblem("IPL_Auction", LpMaximize)

In [8]:
players_lp = LpVariable.dicts("Chosen",players,0,1,cat='Integer')

In [18]:
def filter_lp_dicts(players_lp, criteria_list):
    filter_dict = {}
    for player in criteria_list:
        filter_dict[player] = players_lp[player]
    return filter_dict

In [20]:
batsman_lp = filter_lp_dicts(players_lp, batsman)
bowlers_lp = filter_lp_dicts(players_lp, bowlers)
ar_lp = filter_lp_dicts(players_lp, allrounders)
wk_lp = filter_lp_dicts(players_lp, wks)
indians_lp = filter_lp_dicts(players_lp, indians)
overseas_lp = filter_lp_dicts(players_lp, overseas)

In [21]:
def constructLpSumArr(dict_lp):
    lp_arr = []
    for k,v in dict_lp.items():
        lp_arr += lpSum(dict_lp[k])
    return lp_arr

In [22]:
rewards = []
costs = []
MAX_AVAILABLE_COST = 30
for k, v in players_lp.items():
    costs += lpSum(base_prices[k] * players_lp[k])
    rewards += lpSum(median_ratings[k] * players_lp[k])
prob+= lpSum(rewards)
prob += lpSum(costs) <= MAX_AVAILABLE_COST
prob += lpSum(constructLpSumArr(wk_lp)) == 1
prob += lpSum(constructLpSumArr(players_lp)) == 11
prob += lpSum(constructLpSumArr(batsman_lp)) >= 3
prob += lpSum(constructLpSumArr(batsman_lp)) <= 5
prob += lpSum(constructLpSumArr(bowlers_lp)) >= 3
prob += lpSum(constructLpSumArr(bowlers_lp))<= 5
prob += lpSum(constructLpSumArr(ar_lp)) >= 1
prob += lpSum(constructLpSumArr(ar_lp)) <= 3
prob += lpSum(constructLpSumArr(indians_lp)) == 7
prob += lpSum(constructLpSumArr(overseas_lp)) == 4

In [24]:
prob.solve()

1

In [27]:
prob

IPL_Auction:
MAXIMIZE
80.34482758620689*Chosen_ABdeVilliers + 21.6990099009901*Chosen_AaronFinch + 2.4724637681159423*Chosen_AbhishekJhunjhunwala + 2.4806451612903233*Chosen_AbhishekNayar + 110.38074712643677*Chosen_AdamGilchrist + 6.0777483124397325*Chosen_AjanthaMendis + 80.23255813953489*Chosen_AjinkyaRahane + 22.91979913916786*Chosen_AjitAgarkar + 2.467512274959084*Chosen_AkshdeepNath + 84.85454545454544*Chosen_AlbieMorkel + 68.15501165501165*Chosen_AmbatiRayudu + 84.19490254872564*Chosen_AmitMishra + 96.0*Chosen_AndreRussell + 32.072463768115945*Chosen_AndrewSymonds + 56.149253731343286*Chosen_AndrewTye + 23.54479166666667*Chosen_AngeloMathews + 121.0*Chosen_AnilKumble + 16.895238095238096*Chosen_AnkitRajpoot + 2.4800000000000004*Chosen_AnkitSharma + 2.4740740740740743*Chosen_AnureetSingh + 48.16391257995736*Chosen_AshishNehra + 9.706684825785949*Chosen_AshishReddy + 38.542386671832624*Chosen_AshokDinda + 83.7125*Chosen_AxarPatel + 94.5*Chosen_AzharMahmood + 2.470921985815603*Chos

In [25]:
def summary(prob):
    div = '---------------------------------------\n'
    print("Variables:\n")
    score = str(prob.objective)
    constraints = [str(const) for const in prob.constraints.values()]
    for v in prob.variables():
        score = score.replace(v.name, str(v.varValue))
        constraints = [const.replace(v.name, str(v.varValue)) for const in constraints]
        if v.varValue != 0:
            print(v.name, "=", v.varValue)
    print(div)
    print("Constraints:")
    for constraint in constraints:
        constraint_pretty = " + ".join(re.findall("[0-9\.]*\*1.0", constraint))
        if constraint_pretty != "":
            print("{} = {}".format(constraint_pretty, eval(constraint_pretty)))
    print(div)
    print("Score:")
    score_pretty = " + ".join(re.findall("[0-9\.]+\*1.0", score))
    print("{} = {}".format(score_pretty, eval(score)))

In [26]:
summary(prob)

Variables:

Chosen_AnilKumble = 1.0
Chosen_BhuvneshwarKumar = 1.0
Chosen_ChrisGayle = 1.0
Chosen_DavidWarner = 1.0
Chosen_HarbhajanSingh = 1.0
Chosen_JacquesKallis = 1.0
Chosen_MunafPatel = 1.0
Chosen_RashidKhan = 1.0
Chosen_RishabhPant = 1.0
Chosen_SureshRaina = 1.0
Chosen_VirenderSehwag = 1.0
---------------------------------------

Constraints:
2.0*1.0 + 2.0*1.0 + 2.0*1.0 + 2.0*1.0 + 2.0*1.0 + 1.5*1.0 + 2.0*1.0 + 2.0*1.0 + 2.0*1.0 + 2.0*1.0 + 2.0*1.0 = 21.5
---------------------------------------

Score:
121.0*1.0 + 120.66666666666669*1.0 + 120.4761904761905*1.0 + 120.55555555555556*1.0 + 120.49107142857143*1.0 + 97.33333333333331*1.0 + 120.3846153846154*1.0 + 121.66666666666669*1.0 + 110.96428571428572*1.0 + 120.51315789473685*1.0 + 120.58823529411764*1.0 = 1294.6397784147396


In [ ]:
prob